Consider GPT-2 XL, which has the following configuration:
-  vocab_size: 50,257
-   context_length: 1,024
-   num_layers: 48
-   d_model: 1,600
-   num_heads: 25
-   d_ff: 6,400

Suppose we constructed our model using this configuration. How many trainable parameters would our model have? Assuming each parameter is represented using singl-precision floating point, how much memory is required to just load this model?

In [204]:
vocab_size = 50257
context_len = 1024
num_layers = 48
d_model = 1600
num_heads = 25
d_ff = 6400

token_embeddings = vocab_size * d_model
position_embeddings = context_len * d_model
embd_layer = token_embeddings + position_embeddings

q_k_v_o = 4 * d_model * d_model
ffn = 2 * d_model * d_ff
transformer_blocks = num_layers * (q_k_v_o + ffn)

lm_head = d_model * vocab_size

total_params = embd_layer + transformer_blocks + lm_head
print(f"Total trainable params {total_params/10**9:0.2f}B")
bytes_float32 = total_params * 4
mem_fp32 = bytes_float32 / 1024 ** 3
bytes_bf16 = total_params * 2
mem_bf16 = bytes_bf16 / 1024 ** 3
print(f"Memory fp32: {mem_fp32:,.2f} GiB")
print(f"Memory bf16: {mem_bf16:,.2f} GiB")

Total trainable params 1.64B
Memory fp32: 6.10 GiB
Memory bf16: 3.05 GiB


Identify the matrix multiplies required to complete a forward pass of our GPT-2 XL-shaped model.
How many FLOPs do these matrix multiplies require in total? Assume that our input
sequence has `context_length` tokens

In [205]:
def get_overall_flops(
    vocab_size,
    context_length,
    num_layers,
    d_model,
    num_heads,
    d_ff,
    b
):
    F_forward_pass = get_forward_flops_TransformerLM(
        vocab_size, context_length, num_layers, d_model, num_heads, d_ff, b
    )
    F_adamw = get_adamw_flops(
        vocab_size,
        context_length,
        num_layers,
        d_model,
        d_ff
    )
    F_backward_pass = get_backward_flops_TransformerLM(
        F_forward_pass
    )
    return F_forward_pass + F_backward_pass + F_adamw

def get_forward_flops_TransformerLM(
    vocab_size,
    context_length,
    num_layers,
    d_model,
    num_heads,
    d_ff,
    b
):
    F_transformer_blocks = get_flops_TransformerBlock(
        b, context_length, d_model, num_heads, d_ff
    ) * num_layers
    F_ln_final = get_flops_RMSNorm(b, context_length, d_model)
    F_lm_head = get_flops_lm_head(b, context_length, d_model, vocab_size)
    return F_transformer_blocks + F_ln_final + F_lm_head

def get_adamw_flops(
    vocab_size,
    context_length,
    num_layers,
    d_model,
    d_ff
):
    # beta_1 * m + (1 - beta_1) * grad # 2 FLOPs
    # beta_2 * v + (1 - beta_2) * (grad**2) # 3  FLOPs
    # m / (1 - beta_1**t) # 1 FLOPs
    # v / (1 - beta_2**t) # 1 FLOPs
    #  (torch.sqrt(v) + eps) # 2 FLOPs
    # m / (torch.sqrt(v) + eps) # 1 FLOPs
    # p - lr * step - lr * wd * p # 4 FLOPs

    # TOTAL CONSTANT FLOPs = 16 
    total_params = (
        2 * vocab_size * d_model  # token and lm head embeddings
        + context_length * d_model  # positional embedding
        + num_layers * (4 * d_model**2 + 2 * d_model * d_ff) # Transformer blocks: attention + FFN
    )
    return 16 * total_params

def get_backward_flops_TransformerLM(
    F_forward_pass
):
    F_backward_pass = 2 * F_forward_pass
    return F_backward_pass


def get_flops_lm_head(b, context_length, d_model, vocab_size):
    return 2 * b * context_length * d_model * vocab_size

def get_flops_TransformerBlock(b, context_length, d_model, num_heads, d_ff):
    F_rmsnorm_1 = get_flops_RMSNorm(b, context_length, d_model)
    F_mha = get_flops_MultiHeadAttention(b, context_length, d_model, num_heads)
    F_rmsnorm_2 = F_rmsnorm_1
    F_ffn = get_flops_FFN(b, context_length, d_model, d_ff)
    return F_rmsnorm_1 + F_mha + F_rmsnorm_2 + F_ffn

def get_flops_RMSNorm(b, context_length, d_model):
    return (3*d_model + 2) * b * context_length

def get_flops_MultiHeadAttention(b, context_length, d_model, num_heads):
    F_QKV_linear_proj = get_qkv_proj(b, context_length, d_model)
    F_attention = get_flops_atten_scores(b, context_length, d_model)
    F_softmax = get_flops_Softmax(b, context_length, num_heads)
    F_weighted = get_flops_atten_scores(b, context_length, d_model)
    F_out_proj = get_flops_output_proj(b, context_length, d_model)
    return F_QKV_linear_proj + F_attention + F_softmax + F_weighted + F_out_proj

def get_qkv_proj(b, context_length, d_model):
    return 3 * (2 * b * context_length * (d_model**2))

def get_flops_atten_scores(b, context_length, d_model):
    return 2 * b * (context_length**2) * d_model

def get_flops_Softmax(b, context_length, num_heads):
    return 5 * b * (context_length**2) * num_heads

def get_flops_output_proj(b, context_length, d_model):
    return 2 * b * context_length * d_model * d_model

def get_flops_FFN(b, context_length, d_model, d_ff):
    F_w1 = 2 * b * context_length * d_model * d_ff
    F_gelu = get_flops_GELU(b, context_length, d_ff)
    F_w2 = F_w1
    return F_w1 + F_gelu + F_w2

def get_flops_GELU(b, context_length, d_ff):
    return 5 * b * context_length * d_ff

def get_table_all_flops(
    vocab_size,
    context_length,
    num_layers,
    d_model,
    num_heads,
    d_ff,
    b = 1
):
    all_model_flops = {
        "Transformer_block": {
            "RMSNorm_1": get_flops_RMSNorm(b, context_length, d_model),
            "MultiHeadAttentio": {
                "QKV_proj": get_qkv_proj(b, context_length, d_model),
                "Attention_scores": get_flops_atten_scores(b, context_length, d_model),
                "Softmax": get_flops_Softmax(b, context_length, num_heads),
                "Attention_weighted": get_flops_atten_scores(b, context_length, d_model),
                "O_proj": get_flops_output_proj(b, context_length, d_model),
                "Overall_MultiHeadAttentio": get_flops_MultiHeadAttention(b, context_length, d_model, num_heads)
            },
            "RMSNorm_2": get_flops_RMSNorm(b, context_length, d_model),
            "FFN": get_flops_FFN(b, context_length, d_model, d_ff),
            "Overall_Transformer_block": get_flops_TransformerBlock(
                b, context_length, d_model, num_heads, d_ff
            ) * num_layers
        },
        "RMSNorm_final": get_flops_RMSNorm(b, context_length, d_model),
        "LM_head": get_flops_lm_head(b, context_length, d_model, vocab_size),
        "forward_pass_TransformerLM": get_forward_flops_TransformerLM(
            vocab_size, context_length, num_layers, d_model, num_heads, d_ff, b
        ),
        "backward_pass_TransformerLM": get_backward_flops_TransformerLM(
            get_forward_flops_TransformerLM(
                vocab_size, context_length, num_layers, d_model, num_heads, d_ff, b)
        ),
        "AdamW-step": get_adamw_flops(
        vocab_size, context_length, num_layers, d_model, d_ff
        ),
        "Overall_TransformerLM": get_overall_flops(
            vocab_size, context_length, num_layers, d_model, num_heads, d_ff, b
        )
    }
    return all_model_flops

In [206]:
import pandas as pd

# --- your FLOP-computing helpers remain unchanged here ------------------ #
# (get_flops_RMSNorm, get_flops_MultiHeadAttention, … get_table_all_flops)
# ----------------------------------------------------------------------- #

def explode_to_tuples(d, prefix=()):
    """Yield (path_tuple, value) pairs from a nested dict."""
    for k, v in d.items():
        path = prefix + (k,)
        if isinstance(v, dict):
            yield from explode_to_tuples(v, path)
        else:
            yield path, v

def flops_table_multiindex(
    vocab_size, context_len, n_layers, d_model, n_heads, d_ff, b=1
):
    # 1. get the nested FLOPs dict
    nested = get_table_all_flops(
        vocab_size, context_len, n_layers, d_model, n_heads, d_ff, b
    )
    overll_flops = nested['Overall_TransformerLM']
    # 2. explode to (tuple, value) pairs
    tuples, vals = zip(*explode_to_tuples(nested))

    # 3. make all tuples the same length
    max_depth = max(len(t) for t in tuples)
    padded = [t + ("",) * (max_depth - len(t)) for t in tuples]

    # 4. build the MultiIndex DataFrame
    names = [f"Level-{i+1}" for i in range(max_depth)]
    mi = pd.MultiIndex.from_tuples(padded, names=names)
    df = pd.DataFrame({"FLOPs": vals}, index=mi)

    # 5. optional pretty formatting
    df["FLOPs_fmt"] = df["FLOPs"].astype(float).map("{:.2e}".format)
    df["FLOPs_proportion"] = df["FLOPs"] / overll_flops
    return df


gpt2_xl = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=48,
    d_model=1600,
    n_heads=25,
    d_ff=6400,
)

gpt2_xl

FLOPs  \
Level-1                     Level-2                   Level-3                                     
Transformer_block           RMSNorm_1                                                   4917248   
                            MultiHeadAttentio         QKV_proj                      15728640000   
                                                      Attention_scores               3355443200   
                                                      Softmax                         131072000   
                                                      Attention_weighted             3355443200   
                                                      O_proj                         5242880000   
                                                      Overall_MultiHeadAttentio     27813478400   
                            RMSNorm_2                                                   4917248   
                            FFN                                                     41975808000   
                            Overall_Transformer_block                             3350357803008   
RMSNorm_final                                                                           4917248   
LM_head                                                                            164682137600   
forward_pass_TransformerLM                                                        3515044857856   
backward_pass_TransformerLM                                                       7030089715712   
AdamW-step                                                                          26192332800   
Overall_TransformerLM                                                            10571326906368   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            4.92e+06   
                            MultiHeadAttentio         QKV_proj                   1.57e+10   
                                                      Attention_scores           3.36e+09   
                                                      Softmax                    1.31e+08   
                                                      Attention_weighted         3.36e+09   
                                                      O_proj                     5.24e+09   
                                                      Overall_MultiHeadAttentio  2.78e+10   
                            RMSNorm_2                                            4.92e+06   
                            FFN                                                  4.20e+10   
                            Overall_Transformer_block                            3.35e+12   
RMSNorm_final                                                                    4.92e+06   
LM_head                                                                          1.65e+11   
forward_pass_TransformerLM                                                       3.52e+12   
backward_pass_TransformerLM                                                      7.03e+12   
AdamW-step                                                                       2.62e+10   
Overall_TransformerLM                                                            1.06e+13   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                4.651496e-07  
                            MultiHeadAttentio         QKV_proj                       1.487859e-03  
                                                      Attention_scores               3.174098e-04  
                                                      Softmax                        1.239882e-05  
                                

In [207]:
gpt2_small = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=12,
    d_model=768,
    n_heads=12,
    d_ff=6400,
)

gpt2_small

FLOPs  \
Level-1                     Level-2                   Level-3                                    
Transformer_block           RMSNorm_1                                                  2361344   
                            MultiHeadAttentio         QKV_proj                      3623878656   
                                                      Attention_scores              1610612736   
                                                      Softmax                         62914560   
                                                      Attention_weighted            1610612736   
                                                      O_proj                        1207959552   
                                                      Overall_MultiHeadAttentio     8115978240   
                            RMSNorm_2                                                  2361344   
                            FFN                                                    20165427200   
                            Overall_Transformer_block                             339433537536   
RMSNorm_final                                                                          2361344   
LM_head                                                                            79047426048   
forward_pass_TransformerLM                                                        418483324928   
backward_pass_TransformerLM                                                       836966649856   
AdamW-step                                                                          3588120576   
Overall_TransformerLM                                                            1259038095360   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            2.36e+06   
                            MultiHeadAttentio         QKV_proj                   3.62e+09   
                                                      Attention_scores           1.61e+09   
                                                      Softmax                    6.29e+07   
                                                      Attention_weighted         1.61e+09   
                                                      O_proj                     1.21e+09   
                                                      Overall_MultiHeadAttentio  8.12e+09   
                            RMSNorm_2                                            2.36e+06   
                            FFN                                                  2.02e+10   
                            Overall_Transformer_block                            3.39e+11   
RMSNorm_final                                                                    2.36e+06   
LM_head                                                                          7.90e+10   
forward_pass_TransformerLM                                                       4.18e+11   
backward_pass_TransformerLM                                                      8.37e+11   
AdamW-step                                                                       3.59e+09   
Overall_TransformerLM                                                            1.26e+12   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                    0.000002  
                            MultiHeadAttentio         QKV_proj                           0.002878  
                                                      Attention_scores                   0.001279  
                                                      Softmax                            0.000050  
                                                 

In [208]:
gpt2_medium = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=24,
    d_model=1024,
    n_heads=16,
    d_ff=6400,
)

gpt2_medium

FLOPs  \
Level-1                     Level-2                   Level-3                                    
Transformer_block           RMSNorm_1                                                  3147776   
                            MultiHeadAttentio         QKV_proj                      6442450944   
                                                      Attention_scores              2147483648   
                                                      Softmax                         83886080   
                                                      Attention_weighted            2147483648   
                                                      O_proj                        2147483648   
                                                      Overall_MultiHeadAttentio    12968787968   
                            RMSNorm_2                                                  3147776   
                            FFN                                                    26876313600   
                            Overall_Transformer_block                             956433530880   
RMSNorm_final                                                                          3147776   
LM_head                                                                           105396568064   
forward_pass_TransformerLM                                                       1061833246720   
backward_pass_TransformerLM                                                      2123666493440   
AdamW-step                                                                          8307376128   
Overall_TransformerLM                                                            3193807116288   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            3.15e+06   
                            MultiHeadAttentio         QKV_proj                   6.44e+09   
                                                      Attention_scores           2.15e+09   
                                                      Softmax                    8.39e+07   
                                                      Attention_weighted         2.15e+09   
                                                      O_proj                     2.15e+09   
                                                      Overall_MultiHeadAttentio  1.30e+10   
                            RMSNorm_2                                            3.15e+06   
                            FFN                                                  2.69e+10   
                            Overall_Transformer_block                            9.56e+11   
RMSNorm_final                                                                    3.15e+06   
LM_head                                                                          1.05e+11   
forward_pass_TransformerLM                                                       1.06e+12   
backward_pass_TransformerLM                                                      2.12e+12   
AdamW-step                                                                       8.31e+09   
Overall_TransformerLM                                                            3.19e+12   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                9.855874e-07  
                            MultiHeadAttentio         QKV_proj                       2.017170e-03  
                                                      Attention_scores               6.723899e-04  
                                                      Softmax                        2.626523e-05  
                                                 

In [209]:
gpt2_large = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=36,
    d_model=1280,
    n_heads=20,
    d_ff=6400,
)

gpt2_large

FLOPs  \
Level-1                     Level-2                   Level-3                                    
Transformer_block           RMSNorm_1                                                  3934208   
                            MultiHeadAttentio         QKV_proj                     10066329600   
                                                      Attention_scores              2684354560   
                                                      Softmax                        104857600   
                                                      Attention_weighted            2684354560   
                                                      O_proj                        3355443200   
                                                      Overall_MultiHeadAttentio    18895339520   
                            RMSNorm_2                                                  3934208   
                            FFN                                                    33587200000   
                            Overall_Transformer_block                            1889654685696   
RMSNorm_final                                                                          3934208   
LM_head                                                                           131745710080   
forward_pass_TransformerLM                                                       2021404329984   
backward_pass_TransformerLM                                                      4042808659968   
AdamW-step                                                                         15291555840   
Overall_TransformerLM                                                            6079504545792   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            3.93e+06   
                            MultiHeadAttentio         QKV_proj                   1.01e+10   
                                                      Attention_scores           2.68e+09   
                                                      Softmax                    1.05e+08   
                                                      Attention_weighted         2.68e+09   
                                                      O_proj                     3.36e+09   
                                                      Overall_MultiHeadAttentio  1.89e+10   
                            RMSNorm_2                                            3.93e+06   
                            FFN                                                  3.36e+10   
                            Overall_Transformer_block                            1.89e+12   
RMSNorm_final                                                                    3.93e+06   
LM_head                                                                          1.32e+11   
forward_pass_TransformerLM                                                       2.02e+12   
backward_pass_TransformerLM                                                      4.04e+12   
AdamW-step                                                                       1.53e+10   
Overall_TransformerLM                                                            6.08e+12   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                6.471264e-07  
                            MultiHeadAttentio         QKV_proj                       1.655781e-03  
                                                      Attention_scores               4.415417e-04  
                                                      Softmax                        1.724772e-05  
                                                 

In [210]:
gpt2_xl = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=48,
    d_model=1600,
    n_heads=25,
    d_ff=6400,
)

gpt2_xl

FLOPs  \
Level-1                     Level-2                   Level-3                                     
Transformer_block           RMSNorm_1                                                   4917248   
                            MultiHeadAttentio         QKV_proj                      15728640000   
                                                      Attention_scores               3355443200   
                                                      Softmax                         131072000   
                                                      Attention_weighted             3355443200   
                                                      O_proj                         5242880000   
                                                      Overall_MultiHeadAttentio     27813478400   
                            RMSNorm_2                                                   4917248   
                            FFN                                                     41975808000   
                            Overall_Transformer_block                             3350357803008   
RMSNorm_final                                                                           4917248   
LM_head                                                                            164682137600   
forward_pass_TransformerLM                                                        3515044857856   
backward_pass_TransformerLM                                                       7030089715712   
AdamW-step                                                                          26192332800   
Overall_TransformerLM                                                            10571326906368   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            4.92e+06   
                            MultiHeadAttentio         QKV_proj                   1.57e+10   
                                                      Attention_scores           3.36e+09   
                                                      Softmax                    1.31e+08   
                                                      Attention_weighted         3.36e+09   
                                                      O_proj                     5.24e+09   
                                                      Overall_MultiHeadAttentio  2.78e+10   
                            RMSNorm_2                                            4.92e+06   
                            FFN                                                  4.20e+10   
                            Overall_Transformer_block                            3.35e+12   
RMSNorm_final                                                                    4.92e+06   
LM_head                                                                          1.65e+11   
forward_pass_TransformerLM                                                       3.52e+12   
backward_pass_TransformerLM                                                      7.03e+12   
AdamW-step                                                                       2.62e+10   
Overall_TransformerLM                                                            1.06e+13   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                4.651496e-07  
                            MultiHeadAttentio         QKV_proj                       1.487859e-03  
                                                      Attention_scores               3.174098e-04  
                                                      Softmax                        1.239882e-05  
                                

In [211]:
gpt2_xl = flops_table_multiindex(
    vocab_size=50257,
    context_len=16384,
    n_layers=48,
    d_model=1600,
    n_heads=25,
    d_ff=6400,
)

gpt2_xl

FLOPs  \
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                   78675968   
                            MultiHeadAttentio         QKV_proj                      251658240000   
                                                      Attention_scores              858993459200   
                                                      Softmax                        33554432000   
                                                      Attention_weighted            858993459200   
                                                      O_proj                         83886080000   
                                                      Overall_MultiHeadAttentio    2087085670400   
                            RMSNorm_2                                                   78675968   
                            FFN                                                     671612928000   
                            Overall_Transformer_block                            132425085616128   
RMSNorm_final                                                                           78675968   
LM_head                                                                            2634914201600   
forward_pass_TransformerLM                                                       135060078493696   
backward_pass_TransformerLM                                                      270120156987392   
AdamW-step                                                                           26585548800   
Overall_TransformerLM                                                            405206821029888   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            7.87e+07   
                            MultiHeadAttentio         QKV_proj                   2.52e+11   
                                                      Attention_scores           8.59e+11   
                                                      Softmax                    3.36e+10   
                                                      Attention_weighted         8.59e+11   
                                                      O_proj                     8.39e+10   
                                                      Overall_MultiHeadAttentio  2.09e+12   
                            RMSNorm_2                                            7.87e+07   
                            FFN                                                  6.72e+11   
                            Overall_Transformer_block                            1.32e+14   
RMSNorm_final                                                                    7.87e+07   
LM_head                                                                          2.63e+12   
forward_pass_TransformerLM                                                       1.35e+14   
backward_pass_TransformerLM                                                      2.70e+14   
AdamW-step                                                                       2.66e+10   
Overall_TransformerLM                                                            4.05e+14   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                1.941625e-07  
                            MultiHeadAttentio         QKV_proj                       6.210612e-04  
                                                      Attention_scores               2.119889e-03  
                                                      Softmax                        8.280816e-05  
               

---

How much peak memory does running AdamW require? Decompose your answer based on the memory usage of the parameters, activations, gradients, and optimizer state. Express your answer
in terms of the batch_size and the model hyperparameters (vocab_size, context_length,
num_layers, d_model, num_heads). Assume d_ff = 4 ×d_model.
For simplicity, when calculating memory usage of activations, consider only the following com-
ponents:
* Transformer block
    - RMSNorm(s)
    - Multi-head self-attention sublayer: QKV projections, QKT matrix multiply, softmax, weighted sum of values, output projection.
    - Position-wise feed-forward: W1 matrix multiply, GELU, W2 matrix multiply
* final RMSNorm
* output embedding
* cross-entropy on logits
Deliverable: An algebraic expression for each of parameters, activations, gradients, and opti-
mizer state, as well as the total.

In [212]:
def get_constant_params(
    vocab_size,
    context_len,
    num_layers,
    d_model,
    d_ff
):
  token_embeddings = vocab_size * d_model
  position_embeddings = context_len * d_model
  embd_layer = token_embeddings + position_embeddings

  # Transformer parameter memory:
  q_k_v_o = 4 * d_model * d_model
  ffn = 2 * d_model * d_ff
  transformer_blocks = num_layers * (q_k_v_o + ffn)

  # lm head parameter:
  lm_head = d_model * vocab_size

  # Total trainable parameters:
  total_params = embd_layer + transformer_blocks + lm_head

  return total_params

def get_activations_per_seq(
    vocab_size,
    context_len,
    num_layers,
    d_model,
    num_heads,
    d_ff,
):
  # RMSNorm
  # Each token goes through this
  rmsnorma_act = context_len * d_model

  # Q, K, V
  # q, k, v projections
  qkv_act = 3 * context_len * d_model

  # scaled dot product attention:
  pre_softmax_act = num_heads * context_len * context_len

  # Concatenate the heads and pass to the next sub-layer:
  attn_weight_act = context_len * d_model

  # FFN:
  # for h1 = W1 @ x -> need to backprop through GELU
  h1_act = context_len * d_ff
  # for h2 = GELU(h1) -> need to backprop through W2
  h2_act = context_len * d_ff
  # total
  ffn_act = h1_act + h2_act

  # Total per-layer activations per sequence
  per_layer_act = rmsnorma_act + qkv_act + pre_softmax_act + attn_weight_act + ffn_act

  # Total for all layers, then add final layer head
  transformer_activations = num_layers * per_layer_act

  final_rmsnorm = context_len * d_model
  logits = context_len * vocab_size

  # Total activations per sequence
  activations_per_sequence = transformer_activations + final_rmsnorm + logits
  return activations_per_sequence


def get_constant_memory(
    vocab_size,
    context_len,
    num_layers,
    d_model,
    d_ff,
    dtype_size
  ):
  # --- Parameter memory ---
  total_params = get_constant_params(
      vocab_size=vocab_size,
      context_len=context_len,
      num_layers=num_layers,
      d_model=d_model,
      d_ff=d_ff,
  )

  # --- Gradients ---:
  gradients = total_params

  # --- Optimizer state (m and v per parameter) --- 
  AdamW_state = 2 * total_params

  # --- Constant Memory ---:
  total_constant = total_params + gradients + AdamW_state
  total_mem_in_bytes = total_constant * dtype_size
  return total_mem_in_bytes

def get_activations_memory(
  vocab_size,
  context_len,
  num_layers,
  d_model,
  num_heads,
  d_ff,
  dtype_size
  ):
  # --- Activation per sequence ---
  activations = get_activations_per_seq(
      vocab_size=vocab_size,
      context_len=context_len,
      num_layers=num_layers,
      d_model=d_model,
      num_heads=num_heads,
      d_ff = d_ff,
  )
  total_mem_in_bytes = activations * dtype_size
  return total_mem_in_bytes

def calc_mem(
    vocab_size,
    context_len,
    num_layers,
    d_model,
    num_heads,
    d_ff = 4*d_model,
    batch_size = 1,
    dtype_size=4
):

  constant_memory = get_constant_memory(
    vocab_size=vocab_size,
    context_len=context_len,
    num_layers=num_layers,
    d_model=d_model,
    d_ff=d_ff,
    dtype_size=dtype_size
  )
  
  activation_memory = get_activations_memory(
    vocab_size,
    context_len,
    num_layers,
    d_model,
    num_heads,
    d_ff,
    dtype_size
  )

  # --- Memory totals in bytes --- 
  total_mem = activation_memory * batch_size + constant_memory
  constant_memory = constant_memory / (1024 ** 3)
  activation_memory = activation_memory / (1024 ** 3)
  total_mem_in_gib = total_mem / (1024 ** 3)

  print(f"Total Constant Memory: {constant_memory:.2f} GiB ({'fp32' if dtype_size==4 else 'bf16'})")
  print(f"Total Activation Memory: {activation_memory:.2f} GiB ({'fp32' if dtype_size==4 else 'bf16'})")
  print(f"Total Memory (a * batch_size + b): {total_mem_in_gib:.2f} GiB ({'fp32' if dtype_size==4 else 'bf16'})")


In [213]:
calc_mem(
    vocab_size=50257,
    context_len=1024,
    num_layers=48,
    d_model=1600,
    d_ff = 4*d_model,
    num_heads=25,
    batch_size=1,
    dtype_size=2
)

Total Constant Memory: 12.20 GiB (bf16)
Total Activation Memory: 4.35 GiB (bf16)
Total Memory (a * batch_size + b): 16.54 GiB (bf16)


Instantiate your answer for a GPT-2 XL-shaped model to get an expression that only depends on
the batch_size. What is the maximum batch size you can use and still fit within 80GB memory?
Deliverable: An expression that looks like a ·batch_size + b for numerical values a, b, and a
number representing the maximum batch size

In [214]:
calc_mem(
    vocab_size=50257,
    context_len=1024,
    num_layers=48,
    d_model=1600,
    d_ff = 4*d_model,
    num_heads=25,
    batch_size=15,
    dtype_size=2
)

Total Constant Memory: 12.20 GiB (bf16)
Total Activation Memory: 4.35 GiB (bf16)
Total Memory (a * batch_size + b): 77.40 GiB (bf16)


How many FLOPs does running one step of AdamW take?

In [231]:
adam_flops = 16 * (
    2 * vocab_size * d_model # tok embedding + lm head
    + context_len * d_model # position embedding
    + num_layers * (4 * d_model**2 + 2 * d_model * d_ff) # Attention + FFN
)
f"{adam_flops/1e12:.4f} TeraFLOPs"

'0.0262 TeraFLOPs'

Model FLOPs utilization (MFU) is defined as the ratio of observed throughput (tokens per second)
relative to the hardware’s theoretical peak FLOP throughput [Chowdhery et al., 2022]. An
NVIDIA A100 GPU has a theoretical peak of 19.5 teraFLOP/s for float32 operations. Assuming
you are able to get 50% MFU, how long would it take to train a GPT-2 XL for 400K steps and a
batch size of 1024 on a single A100? Following Kaplan et al. [2020] and Hoffmann et al. [2022],
assume that the backward pass has twice the FLOPs of the forward pass.

The number of days training would take, with a brief justification.

In [262]:
gpt2_xl = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=48,
    d_model=1600,
    n_heads=25,
    d_ff=6400,
)

total_transformer_flops_per_step = gpt2_xl["FLOPs"]["Overall_TransformerLM"].values[0]
print("Total transformer flops per step:")
print(f"{total_transformer_flops_per_step / 10e12:.4f} TeraFLOPs")
print("Total transformer flops over 400K steps:")
total_transformer_flops = 400*1e3 * total_transformer_flops_per_step
print(f"{total_transformer_flops / 10e12:,.4f} TeraFLOPs")
print("Total training time with NVIDIA A100 GPU:")
seconds = total_transformer_flops / (19.5*1e12 * 0.5)
days = seconds / (60 * 60 * 24)
print(f"{days:.2f} days")


Total transformer flops per step:
1.0571 TeraFLOPs
Total transformer flops over 400K steps:
422,853.0763 TeraFLOPs
Total training time with NVIDIA A100 GPU:
5.02 days
